In [2]:
import datetime as dt
import wget
import os
import pandas as pd
from zipfile import ZipFile

In [3]:
# só extrai o arquivo dado como parâmetro para a pasta ./Dados/temp
def extrai_arquivo(arq):
    try:
        ZipFile(arq, 'r').extractall('./Dados/temp/')
    except:
        print(f"erro ao extrair {arq}")

In [4]:
# baixa dados, concatena em um dataframe só e salva em um .csv
# limpa depois
def baixa_e_concatena(ticker, timeframe, ano_inicial):
    ano_corrente, mes_corrente, dia_corrente = [dt.date.today().year, dt.date.today().month, dt.date.today().day]
    
    # baixa dados da binance conforme ticker e timeframe selecionados para a pasta ./Dados/
    # timeframes disponiveis: 12h 15m 1d 1h 1m 1mo 1s 1w 2h 30m 3d 3m 4h 5m 6h 8h
    # tickers disponiveis: https://data.binance.vision/?prefix=data/spot/monthly/klines/
    url = "https://data.binance.vision/data/spot/monthly/klines/"
    if not os.path.exists(f"./Dados/Processados/{ticker}-{timeframe}.csv"):
        for ano in range(ano_inicial, ano_corrente+1):
            for mes in range(1,12+1):
                mes = str(mes).zfill(2)
                if not ((os.path.exists(f"./Dados/temp/{ticker}-{timeframe}-{ano}-{mes}.zip"))):
                    try:
                        wget.download(f"{url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip"
                                      , out = f"./Dados/temp/")
                        pass
                    except:
                        print(f"\nFalha ao baixar {url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip")
                else:
                    print(f"{ano}/{mes} já baixado")
    else:
        print(f"{ticker}-{timeframe} já processado")
        return
    
    # cria uma lista de arquivos do ticker e timeframe selecionado
    lista_arquivos = os.listdir("./Dados/temp/")
    lista_arquivos = [x for x in lista_arquivos if x.startswith(f"{ticker}-{timeframe}")]
    lista_arquivos[-5:]
    
    # cria um dataframe vazio pra colocar todos os dados dentro
    nomes = ["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume"
                                 ,"Number of trades","Taker buy base asset volume","Taker buy quote asset volume","Ignore"]
    df = pd.DataFrame(columns = nomes)
    
    # concatena tudo em um CSV e deixa na pasta ./Dados/Processados/
    for arq in lista_arquivos:
        extrai_arquivo(f"./Dados/temp/{arq}")
        df = pd.concat([df, pd.read_csv(f'./Dados/temp/{arq[:-4]}.csv', sep=',',decimal='.'
                                   , encoding='latin1', names=nomes, header=None)], ignore_index=True, copy=False)
        os.remove(f"./Dados/temp/{arq[:-4]}.csv")
    df.drop("Ignore", inplace=True, axis=1)
    df.set_index("Open time", inplace=True)
    df.to_csv(f"./Dados/Processados/{ticker}-{timeframe}.csv")
    
    print(f"./Dados/Processados/{ticker}-{timeframe}.csv")
    
    # deleta tudo que é temporario e já foi processado
    for arq in lista_arquivos:
        os.remove(f"./Dados/temp/{arq}")
    
    return

In [5]:
# o de 1s demora muito. Pesa 16GB o arquivo final enquanto o de 1m pesa 300MB. tem que ver se vale a pena mesmo.
ticker = "BTCUSDT"
timeframes = ["1s", "1m", "15m", "30m", "1h", "2h", "4h", "8h", "1d"]
for timeframe in timeframes:
    baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017)

BTCUSDT-1s já processado
BTCUSDT-1m já processado
BTCUSDT-15m já processado
BTCUSDT-30m já processado
BTCUSDT-1h já processado
BTCUSDT-2h já processado
BTCUSDT-4h já processado
BTCUSDT-8h já processado
BTCUSDT-1d já processado


In [84]:
# transforma as timestamps em datetime
df = pd.read_csv(f"./Dados/Processados/BTCUSDT-1h.csv")
df.loc[:,["Open time", "Close time"]] = df[["Open time", "Close time"]].apply(pd.to_datetime, unit="ms")
#df.to_csv("teste.csv", decimal=",", sep=";")
df

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:59:59.999,2.023661e+05,171,35.160503,1.509525e+05
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:59:59.999,1.003048e+05,102,21.448071,9.260828e+04
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:59:59.999,3.128231e+04,36,4.802861,2.079532e+04
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:59:59.999,1.924106e+04,25,2.602292,1.129135e+04
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59.999,4.239504e+03,28,0.814655,3.552747e+03
...,...,...,...,...,...,...,...,...,...,...,...
46233,2022-11-30 19:00:00,16920.45,17147.16,16917.30,17062.85,23959.641700,2022-11-30 19:59:59.999,4.085239e+08,517212,11834.484550,2.017760e+08
46234,2022-11-30 20:00:00,17061.14,17103.52,17030.21,17097.19,10883.041700,2022-11-30 20:59:59.999,1.856127e+08,232781,5482.091140,9.350484e+07
46235,2022-11-30 21:00:00,17098.37,17111.70,17028.37,17106.65,10824.192860,2022-11-30 21:59:59.999,1.847139e+08,221837,5427.768040,9.262509e+07
46236,2022-11-30 22:00:00,17106.64,17249.00,17069.04,17148.29,10013.790860,2022-11-30 22:59:59.999,1.714478e+08,197182,5179.914440,8.870750e+07


In [85]:
df2 = pd.read_csv(f"./Dados/Processados/BTCUSDT-8h.csv")
df2.loc[:,["Open time", "Close time"]] = df2[["Open time", "Close time"]].apply(pd.to_datetime, unit="ms")
df2

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,2017-08-17 00:00:00,4261.48,4349.99,4261.32,4349.99,82.088865,2017-08-17 07:59:59.999,3.531943e+05,334,64.013727,2.756474e+05
1,2017-08-17 08:00:00,4333.32,4485.39,4333.32,4352.34,238.181883,2017-08-17 15:59:59.999,1.056740e+06,1106,183.971766,8.164738e+05
2,2017-08-17 16:00:00,4352.33,4369.69,4200.74,4285.08,474.879629,2017-08-17 23:59:59.999,2.044836e+06,1987,368.263048,1.586095e+06
3,2017-08-18 00:00:00,4285.08,4340.62,4134.61,4300.25,524.582072,2017-08-18 07:59:59.999,2.239760e+06,2171,418.302269,1.787987e+06
4,2017-08-18 08:00:00,4285.00,4371.52,4193.70,4236.89,372.109513,2017-08-18 15:59:59.999,1.595054e+06,1577,314.184044,1.346445e+06
...,...,...,...,...,...,...,...,...,...,...,...
5788,2022-11-29 08:00:00,16463.95,16548.71,16328.84,16393.48,102728.309540,2022-11-29 15:59:59.999,1.690793e+09,1999312,51373.160530,8.455401e+08
5789,2022-11-29 16:00:00,16394.34,16526.91,16342.59,16442.53,57908.988240,2022-11-29 23:59:59.999,9.513885e+08,1281999,28705.228560,4.716029e+08
5790,2022-11-30 00:00:00,16442.91,17162.20,16428.30,16884.18,109582.137730,2022-11-30 07:59:59.999,1.847274e+09,2292405,55770.095240,9.400179e+08
5791,2022-11-30 08:00:00,16883.87,16930.66,16766.68,16865.64,78269.022140,2022-11-30 15:59:59.999,1.319176e+09,1768028,38557.684210,6.498694e+08


In [86]:
# une os 2 DF's, mas fica cheio de Nan's arrumar isso ou achar outro metodo
# outros metodos:  pd.grouper, aggregate ou resampler
pd.merge(df, df2, on="Open time", how="left")

,Open time,Open_x,High_x,Low_x,Close_x,Volume_x,Close time_x,Quote asset volume_x,Number of trades_x,Taker buy base asset volume_x,...,Open_y,High_y,Low_y,Close_y,Volume_y,Close time_y,Quote asset volume_y,Number of trades_y,Taker buy base asset volume_y,Taker buy quote asset volume_y
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:59:59.999,2.023661e+05,171,35.160503,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:59:59.999,1.003048e+05,102,21.448071,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:59:59.999,3.128231e+04,36,4.802861,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:59:59.999,1.924106e+04,25,2.602292,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59.999,4.239504e+03,28,0.814655,...,4333.32,4485.39,4333.32,4352.34,238.181883,2017-08-17 15:59:59.999,1.056740e+06,1106.0,183.971766,816473.809215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46233,2022-11-30 19:00:00,16920.45,17147.16,16917.30,17062.85,23959.641700,2022-11-30 19:59:59.999,4.085239e+08,517212,11834.484550,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
46234,2022-11-30 20:00:00,17061.14,17103.52,17030.21,17097.19,10883.041700,2022-11-30 20:59:59.999,1.856127e+08,232781,5482.091140,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
46235,2022-11-30 21:00:00,17098.37,17111.70,17028.37,17106.65,10824.192860,2022-11-30 21:59:59.999,1.847139e+08,221837,5427.768040,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
46236,2022-11-30 22:00:00,17106.64,17249.00,17069.04,17148.29,10013.790860,2022-11-30 22:59:59.999,1.714478e+08,197182,5179.914440,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
